In [ ]:
!pip install scikit-plot
!pip install jax-unirep

Try to deal with GPU memory 
accroading to https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html

In [2]:
import os

os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='False'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']='.95'
os.environ['XLA_PYTHON_CLIENT_ALLOCATOR']='platform'

import jax
# Global flag to set a specific platform, must be used at startup.
jax.config.update('jax_platform_name', 'cpu')

import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

from Bio import SeqIO
from jax_unirep import get_reps
from jax_unirep import evotune, fit
from jax_unirep.utils import dump_params

In [3]:
fastas = "/home/ubuntu/data/bk_fasta/SRR12103592.assembly.len10.fasta"
# plk = "/home/kongkitimanonk/SCRATCH_NOBAK/phase3/bat_blood.len10.pkl"

In [7]:
reset_device_memory(True)
clear_jax_caches()

In [2]:
import gc
import jax 

def clear_jax_caches():
  """Utility to clear all the function caches in jax."""
  # main jit/pmap lu wrapped function caches - have to grab from closures
  jax.xla._xla_callable.__closure__[1].cell_contents.clear()
  jax.pxla.parallel_callable.__closure__[1].cell_contents.clear()
  # primitive callable caches
  jax.xla.xla_primitive_callable.cache_clear()
  jax.xla.primitive_computation.cache_clear()
  # jaxpr caches for control flow and reductions
  jax.lax.lax_control_flow._initial_style_jaxpr.cache_clear()
  jax.lax.lax_control_flow._fori_body_fun.cache_clear()
  jax.lax.lax._reduction_jaxpr.cache_clear()
  # these are trivial and only included for completeness sake
  jax.lax.lax.broadcast_shapes.cache_clear()
  jax.xla.xb.get_backend.cache_clear()
  jax.xla.xb.dtype_to_etype.cache_clear()
  jax.xla.xb.supported_numpy_dtypes.cache_clear()
    
def reset_device_memory(delete_objs=True):
    """Free all tracked DeviceArray memory and delete objects.

  Args:
    delete_objs: bool: whether to delete all live DeviceValues or just free.

  Returns:
    number of DeviceArrays that were manually freed.
  """
    dvals = (x for x in gc.get_objects() if isinstance(x, jax.xla.DeviceArray))
    n_deleted = 0
    for dv in dvals:
    
        if not isinstance(dv, jax.xla.DeviceConstant):
            try: 
                dv._check_if_deleted()  # pylint: disable=protected-access
                dv.device_buffer.delete()
                n_deleted += 1
            except:
                pass
        if delete_objs:

            del dv
    del dvals
    gc.collect()
    return n_deleted

In [3]:
def createREPs(df, filename):
    _h_avg, h_final, c_final= get_reps(df['Sequence'].to_list())
    df.drop(columns=['Sequence'], inplace=True)
    df['reps']=_h_avg.tolist() # if there is a problem , might be here , possible solution is reindexing
    df.to_pickle(filename) # dont forget to change the file name to 0_613834

In [5]:
with open(fastas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    seqs = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        # Remove leading and trailing characters from a string
        seqs.append(str(seq_record.seq.strip('*')))
        lengths.append(len(seq_record.seq))

In [6]:
# dictionary of lists  
dict = {'ID': identifiers, 'Sequence': seqs, 'length': lengths}  
df = pd.DataFrame(dict) 
#df["Sequence"] =  seqs
#df.to_pickle(plk)    
df  

,ID,Sequence,length
0,0,MLRVPPAPTP,12
1,1,MELELVAPATSPVPVSFMWS,22
2,2,MNPMRRTVRKASSHTRSTTYRAKCPGLFECWPEREP,38
3,3,MVTSRERGATRLPKEG,18
4,4,MWGPQAALPAWWM,15
...,...,...,...
12569050,12569050,MPILKGLASA,12
12569051,12569051,MCVPPEAARSVEEDDLPPRGEDRSSVKGIRVAALPC,38
12569052,12569052,MPGQGEARGNSGGGP,17
12569053,12569053,MMFSSRQGTALIIIII,18


Remove Duplicate

In [7]:
df.drop_duplicates(subset=['Sequence'],inplace=True)
df

,ID,Sequence,length
0,0,MLRVPPAPTP,12
1,1,MELELVAPATSPVPVSFMWS,22
2,2,MNPMRRTVRKASSHTRSTTYRAKCPGLFECWPEREP,38
3,3,MVTSRERGATRLPKEG,18
4,4,MWGPQAALPAWWM,15
...,...,...,...
12568865,12568865,PGFPDLYEPVMEAIKDLHVPNEDRIKQLLGQDAWTSQKSELAGFYP...,103
12568932,12568932,HNEFFRPYELSSFDDTFCLAMTSSARWNDLGGCVIPTGRLQTGVLR...,97
12568985,12568985,RKTRLTASSRKTATGQTVVLSQPALVQLQASGVLPSPQPVLAVSGG...,229
12569028,12569028,MADLEGDVEVADAFPILVATSC,24


In [8]:
# Setting my chunk size
chunk_size = 5000
# Assigning chunk numbers to rows
df['chunk'] = df['ID'].apply(lambda x: int(int(x)/ chunk_size))
# We don't want the 'chunk' and 'index' columns in the output
cols = [col for col in df.columns if col not in ['chunk']]
# groupby chunk and export each chunk to a different csv.
i = 0
for _, chunk in df.groupby('chunk'):
    chunk[cols].to_csv(f'/mnt/vdb/Bat/chunks/chunk{i}.csv', index=False)
    i += 1
print("complete")

complete


In [3]:
import glob
appended_reps = []
for infile in glob.glob("/mnt/vdb/Bat/reps/*.pkl"):
    # print(infile)
    appended_reps.append(infile)
print(len(appended_reps))

328


In [ ]:
for infile in glob.glob("/mnt/vdb/Bat/chunks/*.csv"):
    #print("Read:"+infile)
    file_name = os.path.basename(infile)
    result="/mnt/vdb/Bat/reps/"+file_name.replace("csv", "pkl")
    if result in appended_reps :
        # print("found then skip : " , result)
        continue
    else:
        df = pd.read_csv(infile) 
        createREPs(df,result)
        print("Save:"+result)
        reset_device_memory()
        clear_jax_caches()
print("complete")

## CD100

In [3]:
fastas = "/home/ubuntu/data/bk_fasta/SRR12103592.assembly.len10.cd100.fasta"
with open(fastas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    seqs = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        # Remove leading and trailing characters from a string
        seqs.append(str(seq_record.seq.strip('*')))
        lengths.append(len(seq_record.seq))
# dictionary of lists  
dict = {'ID': identifiers, 'Sequence': seqs, 'length': lengths}  
df = pd.DataFrame(dict) 
#df["Sequence"] =  seqs
#df.to_pickle(plk)    
df  

,ID,Sequence,length
0,9,EDTGFYPSEPMLCSESEEGQVPHSLETLYQSADCSSPSDALIVCIH...,275
1,19,ALGPSLWDRRRSLHLLLQEAFPVAQSLAQVIHHQFQTVSKQGGPLP,47
2,32,VGICGSDVHYWQHGRIGDFIVKKPMVLGHEASGTVVKVGSLVKHLQ...,151
3,54,EKPCNSNQQPLENLVEDTLINYSQFGSPKDHEHNGCKLCQTDRYCE...,199
4,61,AAKFVFRHNDPDHLEKLLKKSNSETPKIVAFETVHSMDGAICPLEE...,152
...,...,...,...
422791,12568817,GAWTEVGLPSQDVSVASCNCCRRPMHFELMSEWERSYFGNMGPQYV...,51
422792,12568833,LLLPLKVLGFLGGQLSVVVLHQPVHIIIIQLQAMDLEIFSSFAP,45
422793,12568849,GGQTPRTAPNPQNPPLPPFWAWSSQNVQFSFKFLILGAEPLDRFLA...,117
422794,12568850,VEESCTIENNSDSTKPKMAAEVDFGDLELFEAFDHPEESLPKPVHT...,174


In [4]:
NON_CODE = "B|Z|J|U|O|X"
# remove ambigous seqeunce and non-canonical amino acids codes
df = df[~df["Sequence"].str.contains(NON_CODE, regex=True)]
df

,ID,Sequence,length
0,9,EDTGFYPSEPMLCSESEEGQVPHSLETLYQSADCSSPSDALIVCIH...,275
1,19,ALGPSLWDRRRSLHLLLQEAFPVAQSLAQVIHHQFQTVSKQGGPLP,47
2,32,VGICGSDVHYWQHGRIGDFIVKKPMVLGHEASGTVVKVGSLVKHLQ...,151
3,54,EKPCNSNQQPLENLVEDTLINYSQFGSPKDHEHNGCKLCQTDRYCE...,199
4,61,AAKFVFRHNDPDHLEKLLKKSNSETPKIVAFETVHSMDGAICPLEE...,152
...,...,...,...
422791,12568817,GAWTEVGLPSQDVSVASCNCCRRPMHFELMSEWERSYFGNMGPQYV...,51
422792,12568833,LLLPLKVLGFLGGQLSVVVLHQPVHIIIIQLQAMDLEIFSSFAP,45
422793,12568849,GGQTPRTAPNPQNPPLPPFWAWSSQNVQFSFKFLILGAEPLDRFLA...,117
422794,12568850,VEESCTIENNSDSTKPKMAAEVDFGDLELFEAFDHPEESLPKPVHT...,174


In [11]:
# Setting my chunk size
chunk_size = 5000
# Assigning chunk numbers to rows
df['chunk'] = df['ID'].apply(lambda x: int(int(x)/ chunk_size))
# We don't want the 'chunk' and 'index' columns in the output
cols = [col for col in df.columns if col not in ['chunk']]
# groupby chunk and export each chunk to a different csv.
i = 0
for _, chunk in df.groupby('chunk'):
    chunk[cols].to_csv(f'/mnt/vdb/Bat/cd100/chunks/chunk{i}.csv', index=False)
    i += 1
print("complete")

complete


Start create repsssss

In [4]:
import glob
appended_reps = []
for infile in glob.glob("/mnt/vdb/Bat/cd100/reps/*.pkl"):
    # print(infile)
    appended_reps.append(infile)
print(len(appended_reps))

1122


In [5]:
for infile in glob.glob("/mnt/vdb/Bat/cd100/chunks/*.csv"):
    #print("Read:"+infile)
    file_name = os.path.basename(infile)
    result="/mnt/vdb/Bat/cd100/reps/"+file_name.replace("csv", "pkl")
    if result in appended_reps :
        # print("found then skip : " , result)
        continue
    else:
        df = pd.read_csv(infile) 
        createREPs(df,result)
        print("Save:"+result)
        reset_device_memory()
        clear_jax_caches()
print("complete")

Save:/mnt/vdb/Bat/cd100/reps/chunk1167.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk198.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk342.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk1638.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk744.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk1119.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk2479.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk248.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk1031.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk929.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk2415.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk2423.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk1827.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk2461.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk72.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk898.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk1204.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk1498.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk93.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk2240.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk1728.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk378.pkl
Save:/mnt/vdb/Bat/cd100/reps/chunk665.pkl
Save:/mnt/vdb/Bat/cd100

In [6]:
!df -h 

Filesystem      Size  Used Avail Use% Mounted on
udev             16G     0   16G   0% /dev
tmpfs           3.2G  776K  3.2G   1% /run
/dev/vda1        47G   43G  2.7G  95% /
tmpfs            16G     0   16G   0% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
tmpfs            16G     0   16G   0% /sys/fs/cgroup
tmpfs           3.2G     0  3.2G   0% /run/user/1000
/dev/vdb        196G  133G   54G  72% /mnt/vdb


## TransPI

In [3]:
fastas = "/mnt/vdb/Bat/transpi/SRR12103592.combined.okay.fa.transdecoder.pep"
with open(fastas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    seqs = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        # Remove leading and trailing characters from a string
        seqs.append(str(seq_record.seq.strip('*')))
        lengths.append(len(seq_record.seq))
# dictionary of lists  
dict = {'ID': identifiers, 'Sequence': seqs, 'length': lengths}  
df = pd.DataFrame(dict) 
#df["Sequence"] =  seqs
#df.to_pickle(plk)    
df  

,ID,Sequence,length
0,SOAP.k25.C303246.p1,QKVLQAAGPSTTTETETIAKYEIMDGAPVKGESIPIRLFLAGYDPT...,101
1,SOAP.k25.C303708.p1,PITWGRKWNIENGCARTHSQDDYSPGSQAQGESGTASHPRRGHLEM...,102
2,SOAP.k25.C304032.p1,LRNHSPLMSFGASFVSFLNAMMTFEEEKMQLACDDLRTTEKLCESE...,103
3,SOAP.k25.C304284.p1,ESTDQISPYGNSTVTQPSDSGWQYNETHTSLKQNTPRNTSKLYIGL...,104
4,SOAP.k25.C304386.p1,LGPDLSWAWEAKQPWGQETSLRRGEGSGLCKVGGVRVCAPPLLTPK...,104
...,...,...,...
11000,Velvet.k61.NODE_783_length_328_cov_36.378048.p1,MSPSQAVYIVPSKGRLIGGLRDTPSYEHFQEDFSTCSLCTFRDLCA...,106
11001,Velvet.k61.NODE_8470_length_279_cov_15.172043.p1,QDLENAATGDAAVHQRIASLPVEVQEVSLLDKITDKESGERLSKMV...,105
11002,Velvet.k61.NODE_9141_length_765_cov_44.260132.p2,ARAALAMPVKGGTKCIKYLLLGFNFVFWLAGIAVLAIGLWLRFDSQ...,233
11003,Velvet.k61.NODE_9415_length_1250_cov_53.216000.p1,MSCKPQCSLNHLPTPCARQSAPFRIPAEFLYLVLLLVEGAPFSNFS...,367


In [4]:
NON_CODE = "B|Z|J|U|O"
# remove ambigous seqeunce and non-canonical amino acids codes
df = df[~df["Sequence"].str.contains(NON_CODE, regex=True)]
df

,ID,Sequence,length
0,SOAP.k25.C303246.p1,QKVLQAAGPSTTTETETIAKYEIMDGAPVKGESIPIRLFLAGYDPT...,101
1,SOAP.k25.C303708.p1,PITWGRKWNIENGCARTHSQDDYSPGSQAQGESGTASHPRRGHLEM...,102
2,SOAP.k25.C304032.p1,LRNHSPLMSFGASFVSFLNAMMTFEEEKMQLACDDLRTTEKLCESE...,103
3,SOAP.k25.C304284.p1,ESTDQISPYGNSTVTQPSDSGWQYNETHTSLKQNTPRNTSKLYIGL...,104
4,SOAP.k25.C304386.p1,LGPDLSWAWEAKQPWGQETSLRRGEGSGLCKVGGVRVCAPPLLTPK...,104
...,...,...,...
11000,Velvet.k61.NODE_783_length_328_cov_36.378048.p1,MSPSQAVYIVPSKGRLIGGLRDTPSYEHFQEDFSTCSLCTFRDLCA...,106
11001,Velvet.k61.NODE_8470_length_279_cov_15.172043.p1,QDLENAATGDAAVHQRIASLPVEVQEVSLLDKITDKESGERLSKMV...,105
11002,Velvet.k61.NODE_9141_length_765_cov_44.260132.p2,ARAALAMPVKGGTKCIKYLLLGFNFVFWLAGIAVLAIGLWLRFDSQ...,233
11003,Velvet.k61.NODE_9415_length_1250_cov_53.216000.p1,MSCKPQCSLNHLPTPCARQSAPFRIPAEFLYLVLLLVEGAPFSNFS...,367


In [7]:
#df["Sequence"] = df["Sequence"].str.replace("*","") # match 
len(df[df["Sequence"].str.contains("*", regex=False)])

0

In [7]:
df.length.describe()

count    11005.000000
mean       214.250159
std        141.477705
min         85.000000
25%        123.000000
50%        165.000000
75%        254.000000
max       2406.000000
Name: length, dtype: float64

In [8]:
# Setting my chunk size
chunk_size = 2000
# Assigning chunk numbers to rows
df['chunk'] = df.index.map(lambda x: int(int(x)/ chunk_size))
# We don't want the 'chunk' and 'index' columns in the output
cols = [col for col in df.columns if col not in ['chunk']]
# groupby chunk and export each chunk to a different csv.
i = 0
for _, chunk in df.groupby('chunk'):
    chunk[cols].to_csv(f'/mnt/vdb/Bat/transpi/chunks/chunk{i}.csv', index=False)
    i += 1
print("complete")

complete


In [9]:
import glob
appended_reps = []
for infile in glob.glob("/mnt/vdb/Bat/transpi/reps/*.pkl"):
    # print(infile)
    appended_reps.append(infile)
print(len(appended_reps))

0


In [10]:
for infile in glob.glob("/mnt/vdb/Bat/transpi/chunks/*.csv"):
    #print("Read:"+infile)
    file_name = os.path.basename(infile)
    result="/mnt/vdb/Bat/transpi/reps/"+file_name.replace("csv", "pkl")
    if result in appended_reps :
        # print("found then skip : " , result)
        continue
    else:
        df = pd.read_csv(infile) 
        createREPs(df,result)
        print("Save:"+result)
        reset_device_memory()
        clear_jax_caches()
print("complete")

Save:/mnt/vdb/Bat/transpi/reps/chunk5.pkl
Save:/mnt/vdb/Bat/transpi/reps/chunk2.pkl
Save:/mnt/vdb/Bat/transpi/reps/chunk3.pkl
Save:/mnt/vdb/Bat/transpi/reps/chunk1.pkl
Save:/mnt/vdb/Bat/transpi/reps/chunk0.pkl
Save:/mnt/vdb/Bat/transpi/reps/chunk4.pkl
complete
